<a href="https://colab.research.google.com/github/auhcheng/mackay-information-theory/blob/master/bent_coin_compressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from numpy.random import default_rng
rng = default_rng()

In [0]:
def create_file(N=10_000, f=0.01):
  string = ''
  for bit in rng.binomial(1, f, N):
    string += str(bit)
  return string
file = create_file()

In [0]:
def compress(string):
  counts = []
  count = 0
  for bit in string:
    if bit == '0':
      count += 1
    if bit == '1':
      counts.append(count)
      count = 0
  counts.append(count+len(string))

  compressed = ''
  for c in counts[:-1]:
    compressed += '{0:014b}'.format(c)
  compressed += '{0:015b}'.format(counts[-1])

  return compressed

In [0]:
def uncompress(compressed):
  counts = []
  r = range(0, len(compressed), 14)
  for i in r[:-2]:
    counts.append(int(compressed[i:i+14], 2))
  counts.append(int(compressed[r[-2]:],2))

  uncompressed = ''
  for c in counts[:-1]:
    uncompressed += '0'*c + '1'
  uncompressed += (counts[-1]-len(string))*'0'
  return uncompressed

In [102]:
string = create_file(f=.01)
print(len(string), string)
compressed = compress(string)
print(len(compressed), compressed)
uncompressed = uncompress(compressed)
print(len(uncompressed), uncompressed)
string == uncompressed

10000 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100001000000000000

True

In [98]:
np.mean([len(compress(create_file())) for _ in range(1000)])/10_000

0.14083779999999999

Each element of the counts list tells you how many 0s you read until the next 1. The last element of the counts list tells you how many 0s you read until the end of line, plus len(string) to differentiate from the previous symbols. If you ever read a number of size len(string) or bigger, you're reading the end of string character.

String -> counts -> numbers in binary (log_2 (10,000) = 13.29 = 14 bits each)

Then to uncompress we can look in chunks of 14 bits each (15 bits for the last one) to extract counts, and then recreate the original file.

On average we get a compression ratio of 14.1%